### Import libraries

In [1]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

### Set the data repository as the working directory 

In [2]:
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\codes


In [3]:
os.chdir("../data")
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\data


### Import the test set (solar and wind inclusion)

##### Test set (solar and wind inclusion)

In [4]:
pd_qd_test_solar_wind_30_pct = pd.read_csv('pd_qd_test_solar_wind_30_pct.csv')
pd_qd_test_solar_wind_30_pct.head()

,scenario,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,1,0.034699,0.007573,0.045639,0.018038,0.041021,0.007125,0.031436,0.009529,0.027359,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,2,0.034556,0.007371,0.045064,0.017669,0.039444,0.007077,0.030837,0.009231,0.026833,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,3,0.034414,0.007166,0.044490,0.017302,0.036141,0.005897,0.030238,0.009051,0.026307,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,4,0.034272,0.006959,0.043915,0.017079,0.034811,0.005607,0.029639,0.008872,0.025781,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,5,0.034130,0.006750,0.043340,0.016992,0.033481,0.005016,0.029040,0.008693,0.025255,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [5]:
pg_v_test_solar_wind_30_pct = pd.read_csv('pg_v_test_solar_wind_30_pct.csv')
pg_v_test_solar_wind_30_pct.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,2.105388e-06,0.352326,1.022860,0.427477,1.004971,0.499906,1.001255,0.500000,1.016953
1,2,2.884193e-07,0.339475,1.022708,0.408073,1.004684,0.499992,1.001383,0.499990,1.017407
2,3,6.290558e-07,0.325290,1.022527,0.386188,1.004354,0.499978,1.001532,0.499987,1.017863
3,4,2.384247e-07,0.312889,1.022411,0.367015,1.004071,0.499990,1.001671,0.499995,1.018324
4,5,2.741106e-07,0.300615,1.022282,0.348093,1.003797,0.499978,1.001810,0.499666,1.018770


### Separate the dependent variables (optimal generator setpoints) and the independent variables (non-dispatchable loads)
Non-dispatchable loads: $x = (P_{D_{i}},Q_{D_{i}})\forall i\in \Omega_{D}$
<br>
Optimal generator setpoints: $y = (P_{G_{j}},V_{j})\forall j\in \Omega_{G}$

##### Test set (solar and wind inclusion)

In [6]:
x_test_solar_wind_30_pct = pd_qd_test_solar_wind_30_pct.loc[:, pd_qd_test_solar_wind_30_pct.columns != 'scenario']
x_test_solar_wind_30_pct.head()

,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,qd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,0.034699,0.007573,0.045639,0.018038,0.041021,0.007125,0.031436,0.009529,0.027359,0.019239,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,0.034556,0.007371,0.045064,0.017669,0.039444,0.007077,0.030837,0.009231,0.026833,0.018800,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,0.034414,0.007166,0.044490,0.017302,0.036141,0.005897,0.030238,0.009051,0.026307,0.018499,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,0.034272,0.006959,0.043915,0.017079,0.034811,0.005607,0.029639,0.008872,0.025781,0.018130,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,0.034130,0.006750,0.043340,0.016992,0.033481,0.005016,0.029040,0.008693,0.025255,0.017825,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [7]:
y_test_solar_wind_30_pct = pg_v_test_solar_wind_30_pct.loc[:, pg_v_test_solar_wind_30_pct.columns != 'scenario']
y_test_solar_wind_30_pct.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,2.105388e-06,0.352326,1.022860,0.427477,1.004971,0.499906,1.001255,0.500000,1.016953
1,2.884193e-07,0.339475,1.022708,0.408073,1.004684,0.499992,1.001383,0.499990,1.017407
2,6.290558e-07,0.325290,1.022527,0.386188,1.004354,0.499978,1.001532,0.499987,1.017863
3,2.384247e-07,0.312889,1.022411,0.367015,1.004071,0.499990,1.001671,0.499995,1.018324
4,2.741106e-07,0.300615,1.022282,0.348093,1.003797,0.499978,1.001810,0.499666,1.018770


### Test the trained models on the test set (solar and wind inclusion)

In [8]:
model_1 = load_model('model_1.h5')
model_2 = load_model('model_2.h5')
model_3 = load_model('model_3.h5')

##### Using model 1 that was trained using training group 1

In [9]:
y_test_solar_wind_30_pct_pred_1 = model_1.predict(x_test_solar_wind_30_pct)
y_test_solar_wind_30_pct_pred_1 = pd.DataFrame(data = y_test_solar_wind_30_pct_pred_1,
                                 columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_30_pct_pred_1.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000206,0.345587,1.023130,0.430712,1.004909,0.499983,1.001009,0.499929,1.017654
1,0.000206,0.332836,1.022993,0.411231,1.004615,0.499990,1.001135,0.499927,1.018120
2,0.000216,0.318102,1.022841,0.389619,1.004294,0.500002,1.001266,0.499938,1.018618
3,0.000221,0.305838,1.022723,0.370733,1.004027,0.500013,1.001393,0.499521,1.019052
4,0.000194,0.295541,1.022645,0.354946,1.003838,0.500012,1.001494,0.495749,1.019345


In [10]:
pg_v_test_solar_wind_30_pct_pred_1 = pd.concat([pg_v_test_solar_wind_30_pct.scenario, y_test_solar_wind_30_pct_pred_1], axis = 1)
pg_v_test_solar_wind_30_pct_pred_1.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000206,0.345587,1.023130,0.430712,1.004909,0.499983,1.001009,0.499929,1.017654
1,2,0.000206,0.332836,1.022993,0.411231,1.004615,0.499990,1.001135,0.499927,1.018120
2,3,0.000216,0.318102,1.022841,0.389619,1.004294,0.500002,1.001266,0.499938,1.018618
3,4,0.000221,0.305838,1.022723,0.370733,1.004027,0.500013,1.001393,0.499521,1.019052
4,5,0.000194,0.295541,1.022645,0.354946,1.003838,0.500012,1.001494,0.495749,1.019345


In [11]:
pg_v_test_solar_wind_30_pct_pred_1.to_csv('pg_v_test_solar_wind_30_pct_pred_1.csv', index = False, float_format = '%0.20f')

##### Using model 2 that was trained using training group 2

In [12]:
y_test_solar_wind_30_pct_pred_2 = model_2.predict(x_test_solar_wind_30_pct)
y_test_solar_wind_30_pct_pred_2 = pd.DataFrame(data = y_test_solar_wind_30_pct_pred_2,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_30_pct_pred_2.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,-0.000010,0.352540,1.022640,0.427986,1.005006,0.499851,1.001225,0.499772,1.016790
1,-0.000004,0.339919,1.022495,0.408451,1.004726,0.499843,1.001346,0.499770,1.017236
2,0.000003,0.325724,1.022302,0.386608,1.004407,0.499836,1.001488,0.499770,1.017679
3,0.000009,0.313346,1.022176,0.367557,1.004136,0.499778,1.001618,0.499714,1.018125
4,0.000017,0.301337,1.022031,0.349028,1.003883,0.499861,1.001743,0.498648,1.018516


In [13]:
pg_v_test_solar_wind_30_pct_pred_2 = pd.concat([pg_v_test_solar_wind_30_pct.scenario, y_test_solar_wind_30_pct_pred_2], axis = 1)
pg_v_test_solar_wind_30_pct_pred_2.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,-0.000010,0.352540,1.022640,0.427986,1.005006,0.499851,1.001225,0.499772,1.016790
1,2,-0.000004,0.339919,1.022495,0.408451,1.004726,0.499843,1.001346,0.499770,1.017236
2,3,0.000003,0.325724,1.022302,0.386608,1.004407,0.499836,1.001488,0.499770,1.017679
3,4,0.000009,0.313346,1.022176,0.367557,1.004136,0.499778,1.001618,0.499714,1.018125
4,5,0.000017,0.301337,1.022031,0.349028,1.003883,0.499861,1.001743,0.498648,1.018516


In [14]:
pg_v_test_solar_wind_30_pct_pred_2.to_csv('pg_v_test_solar_wind_30_pct_pred_2.csv', index = False, float_format = '%0.20f')

##### Using model 3 that was trained using training group 3

In [15]:
y_test_solar_wind_30_pct_pred_3 = model_3.predict(x_test_solar_wind_30_pct)
y_test_solar_wind_30_pct_pred_3 = pd.DataFrame(data = y_test_solar_wind_30_pct_pred_3,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_30_pct_pred_3.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000420,0.351804,1.022901,0.427387,1.004995,0.500098,1.001266,0.500280,1.016965
1,0.000437,0.339076,1.022768,0.407932,1.004710,0.500105,1.001397,0.500292,1.017411
2,0.000455,0.324828,1.022594,0.386057,1.004378,0.500113,1.001552,0.500282,1.017871
3,0.000469,0.312378,1.022481,0.366939,1.004093,0.500119,1.001694,0.500280,1.018336
4,0.000480,0.300023,1.022354,0.348022,1.003823,0.500130,1.001836,0.500042,1.018782


In [16]:
pg_v_test_solar_wind_30_pct_pred_3 = pd.concat([pg_v_test_solar_wind_30_pct.scenario, y_test_solar_wind_30_pct_pred_3], axis = 1)
pg_v_test_solar_wind_30_pct_pred_3.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000420,0.351804,1.022901,0.427387,1.004995,0.500098,1.001266,0.500280,1.016965
1,2,0.000437,0.339076,1.022768,0.407932,1.004710,0.500105,1.001397,0.500292,1.017411
2,3,0.000455,0.324828,1.022594,0.386057,1.004378,0.500113,1.001552,0.500282,1.017871
3,4,0.000469,0.312378,1.022481,0.366939,1.004093,0.500119,1.001694,0.500280,1.018336
4,5,0.000480,0.300023,1.022354,0.348022,1.003823,0.500130,1.001836,0.500042,1.018782


In [17]:
pg_v_test_solar_wind_30_pct_pred_3.to_csv('pg_v_test_solar_wind_30_pct_pred_3.csv', index = False, float_format = '%0.20f')

##### RMSE of $P_{G_{1}}$ estimates

In [18]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_1, pg_v_test_solar_wind_30_pct_pred_1.pg_1, squared = False)

0.002823704702262387

In [19]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_1, pg_v_test_solar_wind_30_pct_pred_2.pg_1, squared = False)

0.002500604043303642

In [20]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_1, pg_v_test_solar_wind_30_pct_pred_3.pg_1, squared = False)

0.04903261229166688

##### RMSE of $P_{G_{18}}$ estimates

In [21]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_18, pg_v_test_solar_wind_30_pct_pred_1.pg_18, squared = False)

0.0013470506260969586

In [22]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_18, pg_v_test_solar_wind_30_pct_pred_2.pg_18, squared = False)

0.0029161102502068847

In [23]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_18, pg_v_test_solar_wind_30_pct_pred_3.pg_18, squared = False)

0.0028002803470346845

##### RMSE of $P_{G_{22}}$ estimates

In [24]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_22, pg_v_test_solar_wind_30_pct_pred_1.pg_22, squared = False)

0.0014636168843218582

In [25]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_22, pg_v_test_solar_wind_30_pct_pred_2.pg_22, squared = False)

0.00040174439926938334

In [26]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_22, pg_v_test_solar_wind_30_pct_pred_3.pg_22, squared = False)

0.0029393501525075915

##### RMSE of $P_{G_{25}}$ estimates

In [27]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_25, pg_v_test_solar_wind_30_pct_pred_1.pg_25, squared = False)

6.729677951352003e-05

In [28]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_25, pg_v_test_solar_wind_30_pct_pred_2.pg_25, squared = False)

0.00015866638005948707

In [29]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_25, pg_v_test_solar_wind_30_pct_pred_3.pg_25, squared = False)

0.006227407711319541

##### RMSE of $P_{G_{33}}$ estimates

In [30]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_33, pg_v_test_solar_wind_30_pct_pred_1.pg_33, squared = False)

0.0011643399090104022

In [31]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_33, pg_v_test_solar_wind_30_pct_pred_2.pg_33, squared = False)

0.0003054697183698381

In [32]:
mean_squared_error(pg_v_test_solar_wind_30_pct.pg_33, pg_v_test_solar_wind_30_pct_pred_3.pg_33, squared = False)

0.005402909769043787

##### RMSE of $V_{18}$ estimates

In [33]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_18, pg_v_test_solar_wind_30_pct_pred_1.v_18, squared = False)

0.0005765257716164712

In [34]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_18, pg_v_test_solar_wind_30_pct_pred_2.v_18, squared = False)

0.0005451348576585131

In [35]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_18, pg_v_test_solar_wind_30_pct_pred_3.v_18, squared = False)

0.006393187511290284

##### RMSE of $V_{22}$ estimates

In [36]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_22, pg_v_test_solar_wind_30_pct_pred_1.v_22, squared = False)

0.00023817731072273962

In [37]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_22, pg_v_test_solar_wind_30_pct_pred_2.v_22, squared = False)

0.00017143237979274557

In [38]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_22, pg_v_test_solar_wind_30_pct_pred_3.v_22, squared = False)

0.0028264094476050704

##### RMSE of $V_{25}$ estimates

In [39]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_25, pg_v_test_solar_wind_30_pct_pred_1.v_25, squared = False)

0.00026563891667787055

In [40]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_25, pg_v_test_solar_wind_30_pct_pred_2.v_25, squared = False)

0.0005232004136806098

In [41]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_25, pg_v_test_solar_wind_30_pct_pred_3.v_25, squared = False)

0.007069013675278644

##### RMSE of $V_{33}$ estimates

In [42]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_33, pg_v_test_solar_wind_30_pct_pred_1.v_33, squared = False)

0.0004708761199855219

In [43]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_33, pg_v_test_solar_wind_30_pct_pred_2.v_33, squared = False)

0.00018630238956204837

In [44]:
mean_squared_error(pg_v_test_solar_wind_30_pct.v_33, pg_v_test_solar_wind_30_pct_pred_3.v_33, squared = False)

0.001369483670641164